In [1]:
import os
import pandas as pd
import re
import unicodedata
from docx import Document
import win32com.client
from collections import Counter
import openpyxl 
from datetime import datetime, timedelta
import rasterio
def excel_serial_to_date(val):
    if pd.isna(val) or str(val).strip().upper() in {"", "NA", "NONE"}:
        return "NA"
    val_str = str(val).strip()
    try:
        val_float = float(val_str)
        if val_float > 59:
            base_date = datetime(1899, 12, 30)
            return (base_date + timedelta(days=val_float)).strftime('%d-%m-%Y')
    except:
        pass
    for fmt in ("%d.%m.%y", "%d-%m-%y", "%d/%m/%y", "%d.%m.%Y", "%d-%m-%Y", "%d/%m/%Y"):
        try:
            return datetime.strptime(val_str, fmt).strftime("%d-%m-%Y")
        except:
            continue
    return "NA"
def drop_numeric_columns(df):
    def is_numeric(col):
        try:
            float(col)
            return True
        except ValueError:
            return False
    numeric_cols = [col for col in df.columns if is_numeric(col)]
    return df.drop(columns=numeric_cols)
def make_unique(columns):
    counts = Counter()
    result = []
    for col in columns:
        counts[col] += 1
        if counts[col] > 1:
            result.append(f"{col}_{counts[col]}")
        else:
            result.append(col)
    return result
def extract_station_clean(text):
    text = os.path.splitext(os.path.basename(str(text)))[0]
    text = re.sub(r'[^A-Za-z0-9]', '', text)
    matches = re.findall(r'\d+', text)
    return matches[-1] if matches else text
def clean_lat_long_pair(lat_raw, lon_raw):
    def extract_digits_and_dir(value):
        value = (
            value.replace(',', '.')
                 .replace('v', '.')
                 .replace('"', '')
                 .replace("'", '')
                 .replace("`", '')
                 .replace("°", '')
                 .replace("º", '')
                 .replace("˚", '')
                 .strip()
        )
        value = re.sub(r'[^0-9NSEWnsew. ]+', '', value)
        value = re.sub(r'(?<=\d)[ .]+(?=\d)', '', value)
        value = re.sub(r'[ .]', '', value)
        match = re.match(r'^([NSEWnsew]?)(\d+)([NSEWnsew]?)$', value)
        if not match:
            raise ValueError(f"Invalid coordinate format: {value}")
        prefix, digits, suffix = match.groups()
        direction = (prefix or suffix).upper() if (prefix or suffix) else None
        return digits, direction
    lat_digits, lat_dir = extract_digits_and_dir(lat_raw)
    lon_digits, lon_dir = extract_digits_and_dir(lon_raw)
    if lat_dir not in ['N', 'S']:
        lat_dir = 'N'
    if lon_dir not in ['E', 'W']:
        lon_dir = 'E'
    if lat_dir in ['E', 'W'] and lon_dir in ['E', 'W']:
        print(" Latitude has direction like longitude — correcting to N")
        lat_dir = 'N'
    elif lat_dir in ['N', 'S'] and lon_dir in ['N', 'S']:
        print(" Longitude has direction like latitude — correcting to E")
        lon_dir = 'E'
    return lat_digits + lat_dir, lon_digits + lon_dir
def parse_latitude(lat_str):
    length = len(lat_str)
    meta = lat_str[-1].upper()
    digits = lat_str[:-1]
    deg = min = sec = 0
    if length == 0 + 1:
        if lat_str[0] == '0':
            deg = int(digits[:1])
        else:
            deg = int(digits[:1])
    elif length == 1 + 1:
        if lat_str[0] == '0':
            deg = int(digits[:1])
        else:
            deg = int(digits[:1])
    elif length == 2 + 1:
        if lat_str[0] == '0':
            deg = int(digits[:2])
        else:
            deg = int(digits[:2])
    elif length == 3 + 1:
        if digits[0] in ['0', '1', '2']:
            deg = int(digits[:2])
            min = int(digits[2:3])
            sec = 0
        else:
            deg = int(digits[:1])
            min = int(digits[1:3])
    elif length == 4 + 1:
        if digits[0] in ['0', '1', '2']:
            deg = int(digits[:2])
            min = int(digits[2:4])
            sec = 0
        else:
            deg = int(digits[:1])
            min = int(digits[1:3])
            sec = int(digits[3:4])
    elif length == 5 + 1:
        if digits[0] in ['0', '1', '2']:
            deg = int(digits[:2])
            min = int(digits[2:4])
            sec = int(digits[4:5])
        else:
            deg = int(digits[0])
            min = int(digits[1:3])
            sec = int(digits[3:5])
    elif length == 6 + 1:
        if digits[0] in ['0', '1', '2']:
            deg = int(digits[:2])
            min = int(digits[2:4])
            sec = int(digits[4:6])
        else:
            deg = int(digits[:1])
            min = int(digits[1:3])
            sec = int(digits[3:6])
    elif length == 7 + 1:
        if digits[0] in ['0', '1', '2']:
            deg = int(digits[:2])
            min = int(digits[2:4])
            sec = int(digits[4:7])
        else:
            deg = int(digits[:1])
            min = int(digits[1:5])
            sec = int(digits[5:7])
    else:
        raise ValueError("unsupported")
    if sec > 59:
        if sec <=599:
            sec = sec/10
        else:
            sec = sec/100
    decimal = deg + (min/60) + (sec/3600)
    if meta in ['S', 'W']:
        decimal *=-1
    return round(decimal, 4)
def parse_longitude(lon_str):
    length = len(lon_str)
    meta = lon_str[-1].upper()
    digits = lon_str[:-1]
    deg = min = sec = 0
    if length == 1 + 1:
        if lon_str[0] == '0':
            deg = int(digits[:2])
        else:
            deg = int(digits[:2])
    elif length == 2 + 1:
        if lon_str[0] == '0':
            deg = int(digits[:2])
        else:
            deg = int(digits[:2]) 
    elif length == 3+ 1:
        if lon_str[0] == '0':
            deg = int(digits[:3])
        else:
            deg = int(digits[:2])
    elif length == 4 + 1:
        if lon_str[0] == '0':
            deg = int(digits[:3])
            min = int(digits[3:4])
        else:
            deg = int(digits[:2])
            min = int(digits[2:4])
    elif length == 5 + 1:
        if lon_str[0] == '0':
            deg = int(digits[:3])
            min = int(digits[3:5])
        else:
            deg = int(digits[:2])
            min = int(digits[2:4])
            sec = int(digits[4:5])
    elif length == 6 + 1:
        if lon_str[0] == '0':
            deg = int(digits[:3])
            min = int(digits[3:5])
            sec = int(digits[5:6])
        else:
            deg = int(digits[:2])
            min = int(digits[2:4])
            sec = int(digits[4:6])
    elif length == 7 + 1:
        if lon_str[0] == '0':
            deg = int(digits[:3])
            min = int(digits[3:5])
            sec = int(digits[5:7])
        else:
            deg = int(digits[:2])
            min = int(digits[2:4])
            sec = int(digits[4:7])
    elif length == 8 + 1:
        if lon_str[0] == '0':
            deg = int(digits[:3])
            min = int(digits[3:5])
            sec = int(digits[5:8])
        else:
            deg = int(digits[:2])
            min = int(digits[2:4])
            sec = int(digits[4:8])
    elif length == 9 + 1:
        if lon_str[0] == '0':
            deg = int(digits[:3])
            min = int(digits[3:6])
            sec = int(digits[6:9])
        else:
            deg = int(digits[:2])
            min = int(digits[2:5])
            sec = int(digits[5:9])
    else:
        raise ValueError("unsupported")
    if sec > 59:
        if sec <=599:
            sec = sec/10
        else:
            sec = sec/100
    decimal = deg + (min/60) + (sec/3600)
    if meta in ['S', 'W']:
        decimal *=-1
    return round(decimal, 4)
def normalize_column_name(name):
    name = str(name).strip()
    name = re.sub(r'[^\x20-\x7E]', '', name)
    name = name.upper()
    name = name.replace("º", "O").replace("°", "O").replace("0", "O")
    name = re.sub(r'[^A-Z]', '', name)

    if "LAT" in name:
        return "Latitude"
    elif "LON" in name:
        return "Longitude"
    elif "DATE" in name:
        return "Date"
    elif "TIME" in name:
        return "Time"
    elif any(k in name for k in ["STATION", "STN", "STNO"]):
        return "Station"
    elif "SLNO" in name or name.startswith("SL"):
        return "Serial"
    else:
        return name
def read_doc_table(doc_path):
    print(f" Reading Word file: {doc_path}")
    rows = []
    if doc_path.lower().endswith(".docx"):
        doc = Document(doc_path)
        for table in doc.tables:
            for row in table.rows:
                row_data = [cell.text.strip() for cell in row.cells]
                if any(row_data):
                    rows.append(row_data)
    elif doc_path.lower().endswith(".doc"):
        word = win32com.client.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(doc_path)
        for table in doc.Tables:
            for i in range(1, table.Rows.Count + 1):
                row_data = []
                for j in range(1, table.Columns.Count + 1):
                    try:
                        text = table.Cell(i, j).Range.Text
                        text = text.replace('\r', '').replace('\a', '').strip()
                        row_data.append(text)
                    except:
                        row_data.append("")
                if any(row_data):
                    rows.append(row_data)
        doc.Close(False)
        word.Quit()
    else:
        raise ValueError("Unsupported file format")
    return rows
def read_excel_metadata(excel_path):
    print(f" Reading Excel file: {excel_path}")
    try:
        if excel_path.lower().endswith('.xls'):
            import xlrd
            wb = xlrd.open_workbook(excel_path)
            sheet = wb.sheet_by_index(0)
            rows = []
            for row_idx in range(sheet.nrows):
                rows.append([str(sheet.cell_value(row_idx, col_idx)) 
                           for col_idx in range(sheet.ncols)])
        else:
            import openpyxl
            wb = openpyxl.load_workbook(excel_path, data_only=True)
            sheet = wb.active
            rows = []
            for row in sheet.iter_rows(values_only=True):
                rows.append([str(cell) if cell is not None else "" for cell in row])
        return rows
    except Exception as e:
        print(f" Error reading Excel file: {e}")
        return None
def extract_metadata_from_word_tables(rows):
    df = pd.DataFrame(rows)
    df = df.map(lambda x: re.sub(r'[\r\n\x07]', '', str(x).strip()))
    def is_likely_header(row):
        text = ' '.join(str(x) for x in row)
        return len(re.findall(r'[A-Za-z]{2,}', text)) >= 2
    header_row_idx = next((i for i, row in df.iterrows() if is_likely_header(row)), 0)
    if header_row_idx + 1 < len(df) and is_likely_header(df.iloc[header_row_idx + 1]):
        combined_headers = [
            f"{a} {b}".strip() 
            for a, b in zip(
                df.iloc[header_row_idx].astype(str), 
                df.iloc[header_row_idx + 1].astype(str)
            )
        ]
        df.columns = make_unique([normalize_column_name(c) for c in combined_headers])
        df = df[header_row_idx + 2:].reset_index(drop=True)
    else:
        df.columns = make_unique([normalize_column_name(c) for c in df.iloc[header_row_idx]])
        df = df[header_row_idx + 1:].reset_index(drop=True)
    station_cols = [
        col for col in df.columns 
        if re.search(r'STATION|STN|SAMPLE|LOCATION', col.upper())
    ]    
    if not station_cols:
        station_cols = [col for col in df.columns if len(col) <= 8]
        if not station_cols:
            station_cols = df.columns[:1]
    station_col = station_cols[0]
    df["MatchKey"] = df[station_col].astype(str).apply(extract_station_clean)
    def get_first_valid_col(df, possible_names):
        for name in possible_names:
            for col in df.columns:
                if name in col.upper():
                    return df[col].astype(str).str.strip()
        return pd.Series(["NA"] * len(df), index=df.index)
    df["Latitude"] = get_first_valid_col(df, ["LAT", "LATITUDE"])
    df["Longitude"] = get_first_valid_col(df, ["LON", "LONG", "LONGITUDE"])
    df["Date"] = get_first_valid_col(df, ["DATE", "DT", "SAMPLING DATE"])
    df["Time"] = get_first_valid_col(df, ["TIME", "TM", "SAMPLING TIME"])
    df['Date'] = df['Date'].apply(excel_serial_to_date)
    return df[["MatchKey", "Latitude", "Longitude", "Date", "Time"]]
def extract_metadata_from_excel_table(rows):
    df = pd.DataFrame(rows)
    df = df.map(lambda x: re.sub(r'[\r\n\x07]', '', str(x).strip()))
    def is_excel_header(row):
        text_cells = [str(x).strip() for x in row if str(x).strip()]
        if len(text_cells) < 3:
            return False
        return sum(bool(re.search(r'[A-Za-z]', c)) for c in text_cells) >= 2
    header_row_idx = next((i for i, row in df.iterrows() if is_excel_header(row)), 0)
    next_row = df.iloc[header_row_idx + 1] if header_row_idx + 1 < len(df) else None
    def is_unit_row_with_date_time(row):
        unit_like = lambda val: bool(re.search(r'°|m|\(.*?\)', str(val)))
        label_like = lambda val: bool(re.search(r'[A-Za-z]', str(val)))
        unit_count = sum(unit_like(val) for val in row[:4])
        label_count = sum(label_like(val) for val in row[4:])
        return unit_count >= 2 and label_count >= 1
    if next_row is not None:
        base_row = df.iloc[header_row_idx].fillna("").astype(str)
        second_row = next_row.fillna("").astype(str)
        combined_headers = [
            b if a.strip() == "" else a
            for a, b in zip(base_row, second_row)
        ]
        df.columns = make_unique([normalize_column_name(c) for c in combined_headers])
        df = df[header_row_idx + 2:].reset_index(drop=True)
    else:
        df.columns = make_unique([normalize_column_name(c) for c in df.iloc[header_row_idx]])
        df = df[header_row_idx + 1:].reset_index(drop=True)
    station_cols = [col for col in df.columns if re.search(r'STATION|STN', col.upper())]
    if not station_cols:
        station_cols = df.columns[:1]
    station_col = station_cols[0]
    df["MatchKey"] = df[station_col].astype(str).apply(extract_station_clean)
    def clean_excel_cell(val):
        val = str(val) if val is not None else ""
        val = unicodedata.normalize("NFKD", val)  
        val = val.replace('\xa0', ' ')            
        val = re.sub(r'\s+', ' ', val)            
        return val.strip()                        
    
    def get_first_valid_col(df, possible_names):
        for name in possible_names:
            for col in df.columns:
                if name in col.upper():
                    return df[col].apply(clean_excel_cell)
        return pd.Series(["NA"] * len(df), index=df.index)
    df["Latitude"] = get_first_valid_col(df, ["LAT", "LATITUDE"])
    df["Longitude"] = get_first_valid_col(df, ["LON", "LONG", "LONGITUDE"])
    df["Date"] = get_first_valid_col(df, ["DATE", "DT", "SAMPLING DATE"])
    df["Time"] = get_first_valid_col(df, ["TIME", "TM", "SAMPLING TIME"])
    def excel_serial_to_time(val):
        if pd.isna(val) or str(val).strip() == "":
            return "NA"
        val_str = str(val).strip()
        if "-" in val_str:
            val_str = val_str.split("-")[0].strip()
        if re.match(r'^\d{1,2} \d{1,2}$', val_str):
            h, m = val_str.split()
            return f"{int(h):02}:{int(m):02}"
        if re.match(r'^\d{3,4}$', val_str):
            val_str = val_str.zfill(4)
            return f"{val_str[:2]}:{val_str[2:]}"
        if re.match(r'^\d{1,2}\.\d{1,2}$', val_str):
            h, m = val_str.split('.')
            return f"{int(h):02}:{int(m):02}"
        if re.match(r'^\d{1,2}:\d{2}(:\d{2})?$', val_str):
            return val_str[:5]
        try:
            val_float = float(val_str)
            total_seconds = int(round(val_float * 86400))
            h = total_seconds // 3600
            m = (total_seconds % 3600) // 60
            return f"{h:02}:{m:02}"
        except:
            return "NA"
    df["Date"] = df["Date"].apply(excel_serial_to_date)
    df["Time"] = df["Time"].apply(excel_serial_to_time)
    return df[["MatchKey", "Latitude", "Longitude", "Date", "Time"]]
import os
import re
import pandas as pd

def parse_avg_file(file_path, relative_path):
    print(f"\nProcessing: {file_path}")

    column_mapping = {
        "meters": "depSM",
        "decibars": "prDM",
        "temp": "t090C",
        "salinity": "Sal00",
        "ox (ml/l)": "sbeox0ML/L",
        "ox": "sbeox0ML/L",
        "% light": "xmiss",
        "%": "xmiss",
        "sigma-t": "sigma-t00",
        "cond (s/m)": "c0mS/cm",
        "light par": "lightPAR",
        "day hr:min:sec": "time",
        "nbin": "nbin"
    }
    compound_fixes = {
        "ox (ml/l) cond (s/m)": ["ox (ml/l)", "cond (s/m)"],
        "salinity cond (s/m)": ["salinity", "cond (s/m)"],
        "temp salinity": ["temp", "salinity"],
        "sigma-t % light": ["sigma-t", "% light"]
    }
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = [line.strip() for line in f if line.strip()]
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin1') as f:
            lines = [line.strip() for line in f if line.strip()]
            print(f"Used latin1 encoding for {file_path}")

    if not lines or len(lines) < 2:
        print(" Empty or invalid file")
        return None, None
    is_csv_format = lines[0].lower().startswith('"raw","data","file"') or "," in lines[1]
    if is_csv_format:
        header_line_idx = next((i for i, line in enumerate(lines) if "meter" in line.lower()), 1)
        header_line = lines[header_line_idx].lower()
        for bad, good in compound_fixes.items():
            header_line = header_line.replace(bad, ",".join(good))
        raw_headers = [h.strip().strip('"') for h in header_line.split(",")]
        headers = []
        skip_next = False
        for i, h in enumerate(raw_headers):
            if skip_next:
                skip_next = False
                continue
            if i + 1 < len(raw_headers) and re.fullmatch(r"\(.*?\)|%|light", raw_headers[i + 1].lower()):
                headers.append(f"{h} {raw_headers[i + 1]}")
                skip_next = True
            else:
                headers.append(h)
        mapped_columns = [column_mapping.get(h.lower(), h) for h in headers]
        data_lines = lines[header_line_idx + 1:]
        data = []
        for line in data_lines:
            parts = [v.strip() for v in line.split(",")]
            if len(parts) < len(mapped_columns):
                parts += ["NA"] * (len(mapped_columns) - len(parts))
            elif len(parts) > len(mapped_columns):
                parts = parts[:len(mapped_columns)]
            data.append(parts)
        if not data:
            print(" No valid CSV-style data rows")
            return None, None
        data_df = pd.DataFrame(data, columns=mapped_columns)
    else:
        header_line_idx = next(
            (i for i, line in enumerate(lines) if any(h in line.lower() for h in ["meters", "decibars", "salinity", "nbin"])),
            None
        )
        if header_line_idx is None:
            print(" Could not detect header in space-separated format")
            return None, None

        header_line = lines[header_line_idx].lower()
        for bad, good in compound_fixes.items():
            header_line = header_line.replace(bad, "  ".join(good))  
        raw_columns = re.split(r'\s{2,}|\t+', header_line.strip())
        mapped_columns = [column_mapping.get(col.lower(), col) for col in raw_columns]
        data_rows = []
        for line in lines[header_line_idx + 1:]:
            if not line.strip():
                continue
            parts = re.split(r'\s{2,}|\t+', line.strip())
            if len(parts) != len(mapped_columns):
                continue
            data_rows.append(parts)
        if not data_rows:
            print(" No data rows found after header")
            return None, None
        data_df = pd.DataFrame(data_rows, columns=mapped_columns)
    def is_numeric(col):  
        try:
            float(col)
            return True
        except:
            return False
    numeric_cols = [col for col in data_df.columns if is_numeric(col)]
    if numeric_cols:
        data_df = data_df.drop(columns=numeric_cols)
    trace_id = f"AVG_{os.path.splitext(os.path.basename(file_path))[0]}"
    data_df["TraceID"] = trace_id
    data_df["folderpath_filename"] = relative_path
    meta_df = pd.DataFrame([{
        "Ship": "NA",
        "Cruise ID": "NA",
        "TraceID": trace_id,
        "Station": extract_station_clean(file_path),
        "Latitude": "NA",
        "Longitude": "NA",
        "Start Time": "NA",
        "Parameters": ", ".join(data_df.columns),
        "folderpath_filename": relative_path
    }])
    return meta_df, data_df
def find_metadata_file(folder_path, root_folder):
    search_path = folder_path
    while True:
        try:
            files = os.listdir(search_path)
            word_files = [f for f in files if f.lower().endswith(('.doc', '.docx'))]
            if word_files:
                file_path = os.path.join(search_path, word_files[0])
                print(f" Found Word document: {file_path}")
                rows = read_doc_table(file_path)
                if rows:
                    return extract_metadata_from_word_tables(rows), "word"
            excel_files = [f for f in files if f.lower().endswith(('.xls', '.xlsx'))]
            if excel_files:
                file_path = os.path.join(search_path, excel_files[0])
                print(f" Found Excel file: {file_path}")
                rows = read_excel_metadata(file_path)
                if rows:
                    return extract_metadata_from_excel_table(rows), "excel"           
            parent = os.path.dirname(search_path)
            if parent == search_path or parent == root_folder:
                return None, None
            search_path = parent
        except Exception as e:
            print(f" Error scanning {search_path}: {e}")
            return None, None
def process_folder(folder_path, root_folder):
    metadata_df, source_type = find_metadata_file(folder_path, root_folder)
    meta_dfs = []
    data_dfs = []
    for filename in os.listdir(folder_path):
        if not filename.lower().endswith('.avg'):
            continue
        file_path = os.path.join(folder_path, filename)
        relative_path = os.path.relpath(file_path, root_folder).replace("\\", "/")
        meta_df, data_df = parse_avg_file(file_path, relative_path)
        if meta_df is None or data_df is None:
            continue
        if metadata_df is not None:
            station_id = meta_df["Station"].iloc[0]
            metadata_match = metadata_df[metadata_df['MatchKey'] == station_id]
            if not metadata_match.empty:
                for col in ['Latitude', 'Longitude', 'Date', 'Time']:
                    if col in metadata_match.columns:
                        meta_df[col] = metadata_match[col].values[0]
                try:
                    lat_raw = str(meta_df['Latitude'].values[0])
                    lon_raw = str(meta_df['Longitude'].values[0])
                    cleaned_lat, cleaned_lon = clean_lat_long_pair(lat_raw, lon_raw)
                    meta_df['Latitude'] = parse_latitude(cleaned_lat)
                    meta_df['Longitude'] = parse_longitude(cleaned_lon)
                except Exception as e:
                    print(f" Failed to convert lat/lon: {e}")
                    meta_df['Latitude'] = 'NA'
                    meta_df['Longitude'] = 'NA'
            date_str = str(meta_df.get("Date", ["NA"])[0]).strip()
            time_str = str(meta_df.get("Time", ["NA"])[0]).strip()
            def parse_datetime(date_str, time_str):
                if date_str == "NA":
                    return "NA"
                try:
                    date_str = re.sub(r"[./]", "-", date_str.strip())
                    parts = date_str.split("-")
                    if len(parts[2]) == 2:
                        year = int(parts[2])
                        parts[2] = f"19{year}" if year > 50 else f"20{year}"
                        date_str = "-".join(parts)
                    if not time_str or time_str.strip().lower() in {"na", "nan"}:
                        time_str = "00:00"
                    else:
                        time_str = re.split(r"–|-|to", time_str)[0].strip()  # first if range
                        time_str = time_str.replace(" ", ":").replace(".", ":")
                        time_str = re.sub(r":+", ":", time_str)
                        h_m = time_str.split(":")
                        if len(h_m) == 2:
                            h, m = h_m
                            time_str = f"{h.zfill(2)}:{m.zfill(2)}"
                        elif len(h_m) == 1 and h_m[0].isdigit():
                            time_str = f"{h_m[0].zfill(2)}:00"
                        else:
                            time_str = "00:00"
                    dt = datetime.strptime(f"{date_str} {time_str}", "%d-%m-%Y %H:%M")
                    return dt.strftime("%b %d %Y %H:%M:%S")
                except Exception as e:
                    try:
                        dt = datetime.strptime(f"{date_str} 00:00", "%d-%m-%Y %H:%M")
                        return dt.strftime("%b %d %Y 00:00:00")
                    except:
                        print(f" Could not parse StartTime: time data '{date_str} {time_str}' – {e}")
                        return "NA"
            meta_df["Start Time"] = parse_datetime(date_str, time_str)
            for col in ["Date", "Time"]:
                if col in meta_df.columns:
                    meta_df.drop(columns=col, inplace=True)
            station_id = meta_df.get("Station", ["NA"])[0]
            lat_val = str(meta_df.get("Latitude", ["NA"])[0]).replace(" ", "").replace("°", "")
            lon_val = str(meta_df.get("Longitude", ["NA"])[0]).replace(" ", "").replace("°", "")
            cruise_id = meta_df.get("Cruise ID", ["NA"])[0]
            start_time = meta_df.get("Start Time", ["NA"])[0]
            trace_id = f"{cruise_id}_{station_id}_{lat_val}_{lon_val}_{start_time}"
            data_df["TraceID"] = trace_id
            meta_df["TraceID"] = trace_id
        else:
            print(f" No metadata file found for folder {folder_path}")
        meta_dfs.append(meta_df)
        data_dfs.append(data_df)
    return meta_dfs, data_dfs
def map_avg_files(root_folder, output_meta_csv='AVG01_meta.csv', output_data_csv='AVG01_data.csv'):
    all_metadata = []
    all_data = []
    for dirpath, dirnames, filenames in os.walk(root_folder):
        if any(f.lower().endswith(ext) for f in filenames for ext in ['.cnv', '.asc', '.dat']):
            continue 
        meta_dfs, data_dfs = process_folder(dirpath, root_folder)
        all_metadata.extend(meta_dfs)
        all_data.extend(data_dfs)
    if all_metadata:
        meta_table = pd.concat(all_metadata, ignore_index=True)
        meta_table.to_csv(output_meta_csv, index=False, na_rep='NA')
        print(f" Saved merged metadata to {output_meta_csv}")
    if all_data:
        data_table = pd.concat(all_data, ignore_index=True)
        data_table.to_csv(output_data_csv, index=False, na_rep='NA')
        print(f" Saved data to {output_data_csv}")
if __name__ == "__main__":
    folder = r"C:\Users\aishwarya\Videos\Captures\Desktop\NEW1\CMLRE_CTD"
    map_avg_files(folder) 
#--------QC----------
print("\n Running QC on extracted data...")

meta = pd.read_csv('AVG01_meta.csv')
data = pd.read_csv('AVG01_data.csv')
raster = rasterio.open(r"C:\Users\aishwarya\Downloads\ETOPO1_Bed_g_geotiff\ETOPO1_Bed_g_geotiff.tif")
bathymetry = raster.read(1)

def is_at_sea(lat, lon):
    try:
        row, col = raster.index(lon, lat)
        return bathymetry[row, col] < 0
    except:
        return False
# === Profile Envelope QC Range for TEMP (GTSPP)
TEMP_PROFILE_ENVELOPE = [
    {"min_depth": 0, "max_depth": 1100, "min_value": -2.0, "max_value": 40.0},
    {"min_depth": 1100, "max_depth": 3000, "min_value": -1.5, "max_value": 18.0},
]

def get_profile_envelope(depth, envelope_table):
    for layer in envelope_table:
        if layer["min_depth"] <= depth < layer["max_depth"]:
            return layer["min_value"], layer["max_value"]
    return None, None
def profile_envelope_qc(df, depth_col='depSM', param_col='t090C', envelope_table=TEMP_PROFILE_ENVELOPE):
    flags = []
    for _, row in df.iterrows():
        depth = row.get(depth_col)
        value = row.get(param_col)

        if pd.isna(depth) or pd.isna(value):
            flags.append(9)  
            continue

        min_val, max_val = get_profile_envelope(depth, envelope_table)
        if min_val is None:
            flags.append(9)  
        elif min_val <= value <= max_val:
            flags.append(1)  
        else:
            flags.append(4)  
    return pd.Series(flags, name=f'{param_col}_PROFILE_QC')

# === QC 1: Valid datetime
meta['datetime'] = pd.to_datetime(meta['Start Time'], errors='coerce')
meta['DATE_QC'] = pd.to_datetime(meta['Start Time'], errors='coerce').dt.year.gt(1997).map({True: 1, False: 4})

# === QC 2: Valid position
valid_lat = meta['Latitude'].between(-40, 30)
valid_lon = meta['Longitude'].between(20, 160)
meta['POS_QC'] = ((valid_lat & valid_lon)).map({True: 1, False: 4})

# === QC 3: Location at Sea
print(" Checking location at sea...")
meta['SEA_QC'] = meta.apply(lambda row: 1 if is_at_sea(row['Latitude'], row['Longitude']) else 4, axis=1)

# === Combine all three station-level QC tests
meta_valid = meta[(meta['DATE_QC'] == 1) & (meta['POS_QC'] == 1) & (meta['SEA_QC'] == 1)]

# === Filter data to only valid profiles
valid_trace_ids = meta_valid['TraceID'].tolist()
#data = data[data['TraceID'].isin(valid_trace_ids)]

# === Gradient and Spike QC Functions
def gradient_test(series, threshold):
    result = (series - (series.shift(-1) + series.shift(1)) / 2).abs() <= threshold
    return result.map({True: 1, False: 4})

def spike_test(series, threshold):
    part1 = (series - (series.shift(-1) + series.shift(1)) / 2).abs()
    part2 = ((series.shift(-1) - series.shift(1)) / 2).abs()
    result = (part1 - part2) <= threshold
    return result.map({True: 1, False: 4})

# === QC 4–6: Variable-level QC for TEMP and PSAL
if 't090C' in data.columns:
    data['TEMP_QC'] = data['t090C'].between(-2, 40).map({True: 1, False: 4})
    data['TEMP_GRAD_QC'] = gradient_test(data['t090C'], 10.0)
    data['TEMP_SPIKE_QC'] = spike_test(data['t090C'], 2.0)

if 'Sal00' in data.columns:
    data['PSAL_QC'] = data['Sal00'].between(0, 41).map({True: 1, False: 4})
    data['PSAL_GRAD_QC'] = gradient_test(data['Sal00'], 5.0)
    data['PSAL_SPIKE_QC'] = spike_test(data['Sal00'], 0.3)
# === QC 7: Profile Envelope Test
if 'depSM' in data.columns and 't090C' in data.columns:
    data['TEMP_PROFILE_QC'] = profile_envelope_qc(data, depth_col='depSM', param_col='t090C')

#meta.drop(columns=['at_sea'], inplace=True)
meta.to_csv("meta12.csv", index=False,na_rep='NaN')
data.to_csv("data12.csv", index=False,na_rep='NaN')

print(" QC complete. Saved meta12.csv and data12.csv ")


 Found Excel file: C:\Users\aishwarya\data5_table.xlsx
 Reading Excel file: C:\Users\aishwarya\data5_table.xlsx
 Found Word document: C:\Users\aishwarya\Videos\Captures\Desktop\NEW1\CMLRE_CTD\cruise165\Station location.doc
 Reading Word file: C:\Users\aishwarya\Videos\Captures\Desktop\NEW1\CMLRE_CTD\cruise165\Station location.doc
 Found Word document: C:\Users\aishwarya\Videos\Captures\Desktop\NEW1\CMLRE_CTD\cruise165\Station location.doc
 Reading Word file: C:\Users\aishwarya\Videos\Captures\Desktop\NEW1\CMLRE_CTD\cruise165\Station location.doc

Processing: C:\Users\aishwarya\Videos\Captures\Desktop\NEW1\CMLRE_CTD\cruise165\ctd\D63.AVG

Processing: C:\Users\aishwarya\Videos\Captures\Desktop\NEW1\CMLRE_CTD\cruise165\ctd\D64.AVG

Processing: C:\Users\aishwarya\Videos\Captures\Desktop\NEW1\CMLRE_CTD\cruise165\ctd\D65A.AVG

Processing: C:\Users\aishwarya\Videos\Captures\Desktop\NEW1\CMLRE_CTD\cruise165\ctd\D66.AVG

Processing: C:\Users\aishwarya\Videos\Captures\Desktop\NEW1\CMLRE_CTD\crui

C:\Users\aishwarya\AppData\Local\Temp\ipykernel_17848\297439251.py:690: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('AVG01_data.csv')


 Checking location at sea...


C:\Users\aishwarya\anaconda3\Lib\site-packages\rasterio\transform.py:410: RuntimeWarning: invalid value encountered in cast
  new_rows = np.floor(new_rows).astype(dtype="int32")
C:\Users\aishwarya\anaconda3\Lib\site-packages\rasterio\transform.py:411: RuntimeWarning: invalid value encountered in cast
  new_cols = np.floor(new_cols).astype(dtype="int32")


 QC complete. Saved meta12.csv and data12.csv 
